In [7]:
from langchain_cohere import CohereEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma


In [5]:
from dotenv import load_dotenv
import os

load_dotenv()

cohere_api_key = os.getenv("COHERE_API_KEY")


In [2]:
import pandas as pd 
books = pd.read_csv("books_data.csv")

In [3]:
books.head()

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_subtitle,tag_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."


In [ ]:
books["tag_description"].to_csv(
    "tag_description.txt",
    index=False,
    header=False,
    encoding="utf-8"
)


In [ ]:
# raw_doc = TextLoader("tag_description.txt", encoding="utf-8").load()
# text_splitter = CharacterTextSplitter(chunk_size=1000 , chunk_overlap=0 ,separator="\n", keep_separator=False)

In [ ]:
from langchain_core.documents import Document

docs = []
with open("tag_description.txt", encoding="utf-8") as f:
    for line in f:
        
        docs.append(Document(page_content=line.rstrip("\n")))
len(docs)  


5197

In [64]:
docs[100]

Document(metadata={}, page_content='"9780060099459 It\'s Christmas Eve, and Amelia Bedelia has to get the Rogers house ready for a visit from Aunt Myra. But with Amelia Bedelia in charge, this Christmas will be one of the funniest ever!"')

In [89]:
texts = [d.page_content.strip() for d in docs if getattr(d, "page_content", None)]
seen = set()
dedup_texts = []
for t in texts:
    if t and t not in seen:
        dedup_texts.append(t)
        seen.add(t)

print(f"Original: {len(docs)} | After clean+dedup: {len(dedup_texts)}")


Original: 5197 | After clean+dedup: 5197


In [69]:
# cembed=CohereEmbeddings(model="embed-english-v3.0")

# vec_store=Chroma.from_documents(
#     documents=docs_s,
#     embedding=cembed
# )

In [70]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document


In [90]:
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
emb = HuggingFaceEmbeddings(
    model_name=MODEL_NAME,
    model_kwargs={"device": "cpu"},               
    encode_kwargs={"normalize_embeddings": True, "batch_size": 64}
)


In [ ]:
# from langchain_chroma import Chroma
### do not run multiple times
# persist_dir = "./chroma_db_local_v2"

# vectorstore = Chroma.from_texts(
#     texts=dedup_texts,
#     embedding=emb,
#     persist_directory=persist_dir,  
# )


In [ ]:
# vectorstore = Chroma(
#     persist_directory=persist_dir,
#     embedding_function=emb,
# )


In [ ]:
results = vectorstore.similarity_search("A book to teach children about nature", k=5)

results

[Document(id='c0674e8f-4db0-4c62-82e7-2db0d932cd9f', metadata={}, page_content='"9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience."'),
 Document(id='dc235a0b-f95d-4d69-8701-dd3bafde73f1', metadata={}, page_content='"9780786808380 Introduce your babies to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a rare opportunity to expose little ones to a range of images on a single subject, from simple child\'s drawings and abstract art to playful photos. A brief text accompanies each image, introducing the baby to some basic -- and sometimes playful -- information about the subjects."'),
 Document(id='b0f990fd-a662-4d2a-9da5-f237870e1e17', metadata={}, page_content='"9780786808397 Introduce your baby to birds, cats, dogs, 

In [103]:
books[books["isbn13"] == int(results[0].page_content.split()[0].strip('"'))]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_subtitle,tag_description
3747,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,9780786808069 Children will discover the excit...


In [104]:
import pandas as pd

In [107]:
def retrive_recommendations(
        query:str,
        top_k: int =10,
) -> pd.DataFrame:
    res = vectorstore.similarity_search(query , k = 50)

    list_books = []

    for i in range(0,len(res)):
        list_books+=[int(res[i].page_content.strip('"').split()[0])]

    return books[books["isbn13"].isin(list_books)].head(top_k)
    





In [ ]:
retrive_recommendations("A book to teach children about nature") 

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_subtitle,tag_description
59,9780007151240,0007151241,The Family Way,Tony Parsons,Parenthood,http://books.google.com/books/content?id=dJEIx...,It should be the most natural thing in the wor...,2005.0,3.51,400.0,2095.0,The Family Way,9780007151240 It should be the most natural th...
324,9780060959036,0060959037,Prodigal Summer,Barbara Kingsolver,Fiction,http://books.google.com/books/content?id=06IwG...,Barbara Kingsolver's fifth novel is a hymn to ...,2001.0,4.00,444.0,85440.0,Prodigal Summer: A Novel,9780060959036 Barbara Kingsolver's fifth novel...
383,9780061144899,0061144894,When the Heart Waits,Sue Monk Kidd,Religion,http://books.google.com/books/content?id=JlP91...,From the Bestselling Author of The Secret Life...,2006.0,4.17,240.0,2141.0,When the Heart Waits: Spiritual Direction for ...,9780061144899 From the Bestselling Author of T...
404,9780064402453,0064402452,Racso and the Rats of NIMH,Jane Leslie Conly,Juvenile Fiction,http://books.google.com/books/content?id=MgoNv...,"‘Racso, a brash and boastful little rodent, is...",1988.0,3.76,288.0,3231.0,Racso and the Rats of NIMH,"9780064402453 ‘Racso, a brash and boastful lit..."
406,9780064403870,0064403874,"R-T, Margaret, and the Rats of NIMH",Jane Leslie Conly,Juvenile Fiction,http://books.google.com/books/content?id=WTHHH...,"When Margaret and her younger brother, Artie, ...",1991.0,3.52,272.0,631.0,"R-T, Margaret, and the Rats of NIMH",9780064403870 When Margaret and her younger br...
407,9780064404419,0064404412,The Rainbow People,Laurence Yep,Juvenile Fiction,http://books.google.com/books/content?id=5AHwq...,"""Culled from 69 stories collected in a [1930s]...",1992.0,3.75,208.0,202.0,The Rainbow People,"9780064404419 ""Culled from 69 stories collecte..."
416,9780064406925,006440692X,Winter on the Farm,Laura Ingalls Wilder,Juvenile Fiction,http://books.google.com/books/content?id=IvlKH...,The Little House books tell the story of a lit...,1997.0,4.13,32.0,400.0,Winter on the Farm,9780064406925 The Little House books tell the ...
427,9780064434874,0064434877,Christmas in the Big Woods,Laura Ingalls Wilder,Juvenile Fiction,http://books.google.com/books/content?id=FT1Yp...,"Long ago, a little girl named Laura Ingalls li...",1997.0,4.19,32.0,2062.0,Christmas in the Big Woods,"9780064434874 Long ago, a little girl named La..."
429,9780064434980,0064434982,The Deer in the Wood,Laura Ingalls Wilder,Juvenile Fiction,http://books.google.com/books/content?id=V7YDW...,Even the youngest child can enjoy a special ad...,1999.0,4.17,32.0,302.0,The Deer in the Wood,9780064434980 Even the youngest child can enjo...
436,9780064471961,0064471969,Shade's Children (rack),Garth Nix,Juvenile Fiction,http://books.google.com/books/content?id=_jlgl...,The Key to Survival Rests in the Hands of Shad...,1998.0,3.90,345.0,10368.0,Shade's Children (rack),9780064471961 The Key to Survival Rests in the...
